# Plotly Candlestick Chart in Python

In [3]:
import pandas.io.data as web
from datetime import datetime
import numpy as np
import pandas as pd
import plotly.plotly as py

In [4]:
df = web.DataReader("gs", 'yahoo', datetime(2008, 1, 1), datetime(2008, 12, 28))

In [5]:
df.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2008-01-02,214.800003,215.050003,206.600006,207.600006,9573500,185.414549
2008-01-03,208.919998,209.039993,204.139999,204.839996,7751700,182.949492
2008-01-04,201.000000,204.000000,198.289993,199.929993,10157900,178.564202
2008-01-07,200.229996,200.250000,189.990005,194.729996,16044400,173.919910
2008-01-08,195.809998,199.500000,189.000000,189.500000,13553700,169.248825


### Initial candlestick chart

In [6]:
INCREASING_COLOR = '#17BECF'
DECREASING_COLOR = '#7F7F7F'

In [7]:
data = [ dict(
    type = 'candlestick',
    open = df.Open,
    high = df.High,
    low = df.Low,
    close = df.Close,
    x = df.index,
    yaxis = 'y2',
    name = 'GS',
    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

layout=dict()

fig = dict( data=data, layout=layout )

### Create the layout object

In [8]:
fig['layout'] = dict()
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

### Add range buttons

In [9]:
rangeselector=dict(
    visible = True,
    x = 0, y = 0.9,
    bgcolor = 'rgba(150, 200, 250, 0.4)',
    font = dict( size = 13 ),
    buttons=list([
        dict(count=1,
             label='reset',
             step='all'),
        dict(count=1,
             label='1yr',
             step='year',
             stepmode='backward'),
        dict(count=3,
            label='3 mo',
            step='month',
            stepmode='backward'),
        dict(count=1,
            label='1 mo',
            step='month',
            stepmode='backward'),
        dict(step='all')
    ]))
    
fig['layout']['xaxis']['rangeselector'] = rangeselector

### Add moving average

In [10]:
def movingaverage(interval, window_size=10):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [11]:
mv_y = movingaverage(df.Close)
mv_x = list(df.index)

# Clip the ends
mv_x = mv_x[5:-5]
mv_y = mv_y[5:-5]

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )

### Set volume bar chart colors

In [12]:
colors = []

for i in range(len(df.Close)):
    if i != 0:
        if df.Close[i] > df.Close[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

### Add volume bar chart

In [13]:
fig['data'].append( dict( x=df.index, y=df.Volume,                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )

### Add bollinger bands

In [14]:
def bbands(price, window_size=10, num_of_std=5):
    rolling_mean = price.rolling(window=window_size).mean()
    rolling_std  = price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std*num_of_std)
    lower_band = rolling_mean - (rolling_std*num_of_std)
    return rolling_mean, upper_band, lower_band

bb_avg, bb_upper, bb_lower = bbands(df.Close)

fig['data'].append( dict( x=df.index, y=bb_upper, type='scatter', yaxis='y2', 
                         line = dict( width = 1 ),
                         marker=dict(color='#ccc'), hoverinfo='none', 
                         legendgroup='Bollinger Bands', name='Bollinger Bands') )

fig['data'].append( dict( x=df.index, y=bb_lower, type='scatter', yaxis='y2',
                         line = dict( width = 1 ),
                         marker=dict(color='#ccc'), hoverinfo='none',
                         legendgroup='Bollinger Bands', showlegend=False ) )

### Plot

In [16]:
py.iplot( fig, filename = 'candlestick-test-3')

PlotlyError: Invalid 'figure_or_data' argument. Plotly will not be able to properly parse the resulting JSON. If you want to send this 'figure_or_data' to Plotly anyway (not recommended), you can set 'validate=False' as a plot option.
Here's why you're seeing this error:

'visibe' is not allowed in 'rangeselector'

Path To Error: ['layout']['xaxis']['rangeselector']['visibe']

Valid attributes for 'rangeselector' at path ['layout']['xaxis']['rangeselector'] under parents ['figure', 'layout', 'xaxis']:

    ['activecolor', 'bgcolor', 'bordercolor', 'borderwidth', 'buttons',
    'font', 'visible', 'x', 'xanchor', 'y', 'yanchor']

Run `<rangeselector-object>.help('attribute')` on any of the above.
'<rangeselector-object>' is the object at ['layout']['xaxis']['rangeselector']